In [25]:
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql import Window

from pyspark.sql.functions import avg, mean, stddev, udf, col, min, max, round
from pyspark.sql.types import BooleanType, BinaryType, DoubleType, FloatType, IntegerType

from copy import copy, deepcopy

import numpy as np


In [3]:
session = SparkSession.builder.appName(name="AppName").getOrCreate()
sc = session.sparkContext

In [3]:
filepath = "./telco-customer-churn.csv"

In [4]:
df = session.read.csv(filepath, header=True)

In [5]:
print((df.columns))

['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']


In [6]:
print(df.printSchema())

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)

None


In [7]:
def change_type(type_, *cols):
    global df
    
    for c in cols:
        df = df.withColumn(c, col(c).cast(type_))
        
    return df


In [8]:
columns = ["MonthlyCharges", "TotalCharges"]
df = change_type(FloatType(), *columns)

# Queries using the Spark Syntax

In [9]:
df.select(stddev("MonthlyCharges")).show()

+---------------------------+
|stddev_samp(MonthlyCharges)|
+---------------------------+
|          30.09004712627172|
+---------------------------+



In [10]:
avg_df = df.groupBy("gender").avg("MonthlyCharges", "TotalCharges").orderBy("avg(TotalCharges)", ascending=True)
avg_df.show()

+------+-------------------+------------------+
|gender|avg(MonthlyCharges)| avg(TotalCharges)|
+------+-------------------+------------------+
|Female|  65.20424314321728|2283.1909871540843|
|  Male|  64.32748243651142| 2283.407860119869|
+------+-------------------+------------------+



In [11]:
df.groupBy("gender").agg(round(min("TotalCharges"), 1).alias("minimumTotalCharges"), max(
    "TotalCharges").alias("maximum")).sort("gender", ascending=False).show()

+------+-------------------+-------+
|gender|minimumTotalCharges|maximum|
+------+-------------------+-------+
|  Male|               18.8| 8684.8|
|Female|               18.9|8672.45|
+------+-------------------+-------+



# Queries using the SQL Syntax

In [12]:
df.createOrReplaceTempView(name="telco")
session.catalog.listTables()

[Table(name='telco', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [13]:
query = "SELECT gender, COUNT(gender) AS number_of_people FROM telco GROUP BY gender"
session.sql(query).show()

+------+----------------+
|gender|number_of_people|
+------+----------------+
|Female|            3488|
|  Male|            3555|
+------+----------------+



In [14]:
query = "SELECT gender, ROUND(MIN(TotalCharges), 1), MAX(TotalCharges) FROM telco GROUP BY gender"
session.sql(query).show()

+------+---------------------------+-----------------+
|gender|round(min(TotalCharges), 1)|max(TotalCharges)|
+------+---------------------------+-----------------+
|Female|                       18.9|          8672.45|
|  Male|                       18.8|           8684.8|
+------+---------------------------+-----------------+



#  Fetching only the rows where column B has the maximum value if we PARTITION BY A


In [57]:
data = [
    ('a', 5, 'v'),
    ('a', 5, 'v'),
    ('a', 8, 'v'),
    ('e', 7, 'v'),
    ('b', 1, "c"),
    ('b', 3, "c"),
    ('c', 3, "c")
]
df2 = session.createDataFrame(data, ["A", "B", "C"])
df2.show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  a|  5|  v|
|  a|  5|  v|
|  a|  8|  v|
|  e|  7|  v|
|  b|  1|  c|
|  b|  3|  c|
|  c|  3|  c|
+---+---+---+



## SQL statement

In [58]:
df2.createOrReplaceTempView("example")
query = "SELECT * FROM (SELECT *, MAX(B) OVER (PARTITION BY A) AS maxB FROM example) WHERE B = maxB"
session.sql(query).show()

+---+---+---+----+
|  A|  B|  C|maxB|
+---+---+---+----+
|  e|  7|  v|   7|
|  c|  3|  c|   3|
|  b|  3|  c|   3|
|  a|  8|  v|   8|
+---+---+---+----+



## Spark Statement

In [59]:
w = Window.partitionBy("A")
df2.withColumn("maxB", max("B").over(w)).where(col("B") == col("maxB")).show()


+---+---+---+----+
|  A|  B|  C|maxB|
+---+---+---+----+
|  e|  7|  v|   7|
|  c|  3|  c|   3|
|  b|  3|  c|   3|
|  a|  8|  v|   8|
+---+---+---+----+



# `Crosstab`

In [60]:
df2.crosstab(col1='A', col2='B').show()

+---+---+---+---+---+---+
|A_B|  1|  3|  5|  7|  8|
+---+---+---+---+---+---+
|  e|  0|  0|  0|  1|  0|
|  b|  1|  1|  0|  0|  0|
|  a|  0|  0|  2|  0|  1|
|  c|  0|  1|  0|  0|  0|
+---+---+---+---+---+---+



## `Coalesce` / `Repartition`

- Coalesce can **only** minimize the number of partitions, in contrast with repartition.

In [104]:
data = [(10, "blue"),
  (13, "red"),
  (15, "blue"),
  (99, "red"),
  (67, "blue")]
df = session.createDataFrame(data=data).toDF('age', 'color')
df.show()

+---+-----+
|age|color|
+---+-----+
| 10| blue|
| 13|  red|
| 15| blue|
| 99|  red|
| 67| blue|
+---+-----+



In [105]:
df.rdd.glom().collect()

[[Row(age=10, color='blue')],
 [Row(age=13, color='red')],
 [Row(age=15, color='blue')],
 [Row(age=99, color='red'), Row(age=67, color='blue')]]

In [106]:
div_df = df.coalesce(2)
div_df.rdd.getNumPartitions()

2

In [107]:
div_df.rdd.glom().collect()

[[Row(age=10, color='blue'), Row(age=13, color='red')],
 [Row(age=15, color='blue'),
  Row(age=99, color='red'),
  Row(age=67, color='blue')]]

In [114]:
# Default number of partitions 200
print(df.repartition('color').rdd.glom().collect())

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [Row(age=13, color='red'), Row(age=99, color='red')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [Row(age=10, color='blue'), Row(age=15, color='blue'), Row(age=67, color='blue')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


# Real world example


Suppose we have $2\times 10^9$ rows split into $13,000$ partitions, but we need to take only $2,000$ random rows. If we sample, we will still have $13,000$ partitions, so most of the partitions will be empty. Thus, we need to repartition the sample.

In general:
`number_of_partitions = number_of_CPUs in the cluster * 2, 3, 4`

**Why did we use the repartition method instead of coalesce?**

A full data shuffle is an expensive operation for large data sets, but our data puddle is only 2,000 rows. The repartition method returns equal sized text files, which are more efficient for downstream consumers.